# Capstone_project_The Battle of Neighborhoods_Week 1

## 1 Introduction
### 1.1 Background
Toronto is the most populous city in Canada and an international centre of business, finance, arts, and culture. Also, it is one of the most multicultural and cosmopolitan cities in the world.\
New York is the most populous major city in the United States. It has been described as the cultural, financial, and media capital of the world.\
Paris is the capital of France and densely populated. It is also one of major centres of finance, diplomacy, commerce, fashion, science and arts in Europe.
### 1.2 Problem description
These cities share similar characteristics, are multicultural and the most populous and financial capital of their respective countries but how similar they really are? This question can be answer by comparing the neighborhoods and determine how similar or dissimilar these cities are. 
### 1.3 Target audience
People living in New York who are planning to move to a similar city. This comparison will help them to decide if these cities are a good candidate for them to move in.


## 2 Data
First, we will need to know the boroughs and the neighborhoods that exist in each city as well as the latitude and longitude coordinates of each neighborhood.\
New York has a total of 5 boroughs and 306 neighborhoods. Toronto officially has 140 neighbourhoods. Paris has 20 boroughs (arrondissements) and 80 neighborhoods (quartiers). These datasets are free online. Here, are the links:\
New York:\
https://geo.nyu.edu/catalog/nyu_2451_34572 \
Toronto:\
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, \
http://cocl.us/Geospatial_data \
Paris: \
https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e \
Next, we will use Foursquare API to explore the neighborhoods and obtain the venues located near by.
